In [25]:
# Import dependencies.
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision.models import ResNet50_Weights
from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms


import argparse
import os
import time

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

NUM_OUTPUT_LABELS = 133
OUTPUT_MODEL_PATH = "./model/resnet.pth"

def test(model, test_loader, criterion, device):
    '''
        Complete this function that can take a model and a 
        testing data loader and will get the test accuray/loss of the model
        Remember to include any debugging/profiling hooks that you might need
    '''
    print("##########################################")
    print("# Testing Model on Whole Testing Dataset #")
    print("##########################################")

    model.eval()
    running_loss = 0
    running_corrects = 0

    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels).item()

    total_loss = running_loss / len(test_loader.dataset)
    total_acc = running_corrects / len(test_loader.dataset)
    print(f"Testing Accuracy: {100 * total_acc}, Testing Loss: {total_loss}")


def train(model, train_loader, validation_loader, criterion, optimizer, device):
    '''
        Complete this function that can take a model and
        data loaders for training and will get train the model
        Remember to include any debugging/profiling hooks that you might need
    '''

    print("##################################################")
    print("# Training Model on train and validation Dataset #")
    print("##################################################")

    epochs = 2
    best_loss = 1e6
    loss_counter = 0
    image_dataset={'train': train_loader, 'valid': validation_loader}

    for epoch in range(epochs):
        for phase in ['train', 'valid']:
            print(f"Epoch {epoch}, Phase {phase}")
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            running_samples=0

            for step, (inputs, labels) in enumerate(image_dataset[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                
                _,  preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels).item()
                running_samples += len(inputs)

                # Print training status (accuracy and loss)
                accuracy = running_corrects / running_samples
                print("Images [{}/{} ({:.0f}%)] Loss: {:.2f} Accuracy: {}/{} ({:.2f}%) Time: {}".format(
                        running_samples,
                        len(image_dataset[phase].dataset),
                        100.0 * (running_samples / len(image_dataset[phase].dataset)),
                        loss.item(),
                        running_corrects,
                        running_samples,
                        100.0*accuracy,
                        time.asctime() # for measuring time for testing, remove for students and in the formatting
                    )
                )

                #NOTE: Comment lines below to train and test on whole dataset
                if running_samples > (0.2 * len(image_dataset[phase].dataset)):
                    break
            
            epoch_loss = running_loss / running_samples
            epoch_acc = running_corrects / running_samples

            if phase == 'valid':
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                else:
                    loss_counter += 1
        
        if loss_counter == 1:
            break

    return model

def net():
    '''
        Complete this function that initializes your model
        Remember to use a pretrained model
    '''
    model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

    # Freeze Model Parameters -> No need to update the pretrained parameters
    for param in model.parameters():
        param.requires_grad = False
    num_features = model.fc.in_features
    model.fc = nn.Sequential(nn.Linear(num_features, NUM_OUTPUT_LABELS))

    return model


def create_data_loaders(data_path, batch_size):
    '''
    This is an optional function that you may or may not need to implement
    depending on whether you need to use data loaders or not
    '''

    transform = transforms.Compose([
        transforms.Resize(size=(224, 224)),
        transforms.ToTensor(),
    ])

    dataset = datasets.ImageFolder(data_path, transform=transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return data_loader

def save_model(model, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    torch.save(model, output_path)

def get_args():
    parser = argparse.ArgumentParser(description="PyTorch Resnet Training ")
    parser.add_argument(
        "--batch-size",
        type=int,
        default=64,
        metavar="N",
        help="input batch size for training (default: 64)",
    )
    parser.add_argument(
        "--test-batch-size",
        type=int,
        default=32,
        metavar="N",
        help="input batch size for testing (default: 1000)",
    )
    parser.add_argument(
        "--validation-batch-size",
        type=int,
        default=32,
        metavar="N",
        help="input batch size for validation (default: 1000)",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=14,
        metavar="N",
        help="number of epochs to train (default: 14)",
    )
    parser.add_argument(
        "--lr", type=float, default=1.0, metavar="LR", help="learning rate (default: 1.0)"
    )
    args = parser.parse_args()
    return args

In [7]:

# Create data loaders
train_loader = create_data_loaders("./dataset/dogImages/train", 16)
validation_loader = create_data_loaders("./dataset/dogImages/valid", 16)
test_loader = create_data_loaders("./dataset/dogImages/test", 16)

# Initialize a model by calling the net function
model=net()

# Create your loss and optimizer
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

'''
Call the train function to start training your model
Remember that you will need to set up a way to get training data from S3
'''

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Running on Device {device}")



Running on Device cpu


In [8]:
model = train(model, train_loader, validation_loader, loss_criterion, optimizer, device=device)


##################################################
# Training Model on train and validation Dataset #
##################################################
Epoch 0, Phase train
Images [16/6680 (0%)] Loss: 4.98 Accuracy: 0/16 (0.00%) Time: Mon May 20 09:05:33 2024
Images [32/6680 (0%)] Loss: 5.21 Accuracy: 0/32 (0.00%) Time: Mon May 20 09:05:34 2024
Images [48/6680 (1%)] Loss: 5.63 Accuracy: 0/48 (0.00%) Time: Mon May 20 09:05:36 2024
Images [64/6680 (1%)] Loss: 5.77 Accuracy: 0/64 (0.00%) Time: Mon May 20 09:05:37 2024
Images [80/6680 (1%)] Loss: 5.86 Accuracy: 0/80 (0.00%) Time: Mon May 20 09:05:38 2024
Images [96/6680 (1%)] Loss: 6.26 Accuracy: 1/96 (1.04%) Time: Mon May 20 09:05:39 2024
Images [112/6680 (2%)] Loss: 5.60 Accuracy: 2/112 (1.79%) Time: Mon May 20 09:05:41 2024
Images [128/6680 (2%)] Loss: 7.10 Accuracy: 2/128 (1.56%) Time: Mon May 20 09:05:42 2024
Images [144/6680 (2%)] Loss: 6.64 Accuracy: 3/144 (2.08%) Time: Mon May 20 09:05:43 2024
Images [160/6680 (2%)] Loss: 6.95 Acc

In [24]:
save_model(model, OUTPUT_MODEL_PATH)

RuntimeError: File ./model/resnet50.pth cannot be opened.